In [39]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rahulsanghera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def get_genre_prediction(prediction):
    genre_dict =  { 0: 'drama', 1: 'comedy', 2: 'action', 3: 'thriller', 4: 'adventure', 5: 'horror', 6: 'fantasy',7: 'crime',8: 'romance', 9: 'animation'}
    result = []
    y = prediction[0]
    for i in range(len(y)):
        if y[i] == 1:
            result.append(genre_dict[i])
    return {'response': result}

In [3]:
df = pd.read_csv("https://finalfinalfinalproject.s3.us-east-2.amazonaws.com/movie_plots_70kentries.csv", sep=';')

In [5]:
genres = df[['drama','comedy','action','thriller','adventure','horror','fantasy','crime','romance','animation']]

In [6]:
plot_df=df[['plot']]

In [7]:
genre_array=genres.values

In [156]:
tokenizer_pun = nltk.RegexpTokenizer(r"\w+")

In [99]:
plot_df['tokens']= plot_df['plot'].apply(tokenizer_pun.tokenize)

/Users/rahulsanghera/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [216]:
#plot_df

In [257]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rahulsanghera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stop_words = set(stopwords.words('english'))

In [101]:
plot_df['tokens']= plot_df['tokens'].apply(lambda tokens:[w for w in tokens if not w in stop_words])

/Users/rahulsanghera/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [217]:
#plot_df

In [103]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('gutenberg')
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rahulsanghera/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rahulsanghera/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/rahulsanghera/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [104]:
plot_df['tokens']= plot_df['tokens'].apply(lambda tokens: [stemmer.stem(w) for w in tokens])

/Users/rahulsanghera/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [218]:
#plot_df

In [106]:
wordnet_lemmatizer = WordNetLemmatizer()
plot_df['tokens']= plot_df['tokens'].apply(lambda tokens: [wordnet_lemmatizer.lemmatize(w) for w in tokens])

/Users/rahulsanghera/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [219]:
#plot_df['tokens'][0]

In [108]:
MAX_SEQUENCE_LENGTH = 62
#EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=1600000)

In [109]:
tokenizer.fit_on_texts(plot_df['tokens'].values)
word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))

Found 51772 unique tokens.


In [111]:
X = tokenizer.texts_to_sequences(plot_df['tokens'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
#print('Shape of data tensor:', X.shape)

Shape of data tensor: (76807, 62)


In [220]:
#X[1]

In [113]:
Y=genre_array
#Y.shape

(76807, 10)

In [221]:
#Y

In [115]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(61445, 62) (61445, 10)
(15362, 62) (15362, 10)


In [116]:
model = Sequential()
model.add(Dense(units=10000, activation='relu', input_dim=62,))
#model.add(Dense(units=10000, activation='relu', input_dim=62,))
model.add(Dense(units=10, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [117]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10000)             630000    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                100010    
Total params: 730,010
Trainable params: 730,010
Non-trainable params: 0
_________________________________________________________________


In [230]:
model.fit(
    X_train,
    Y_train,
    epochs=10,
    shuffle=True,
    verbose=1
)

Epoch 1/10
1921/1921 [==============================] - 6s 3ms/step - loss: 0.4197 - accuracy: 0.5766
Epoch 2/10
1921/1921 [==============================] - 6s 3ms/step - loss: 0.4186 - accuracy: 0.5766
Epoch 3/10
1921/1921 [==============================] - 6s 3ms/step - loss: 0.4143 - accuracy: 0.5768
Epoch 4/10
1921/1921 [==============================] - 6s 3ms/step - loss: 0.4140 - accuracy: 0.5769
Epoch 5/10
1921/1921 [==============================] - 7s 3ms/step - loss: 0.4141 - accuracy: 0.5769
Epoch 6/10
1921/1921 [==============================] - 7s 4ms/step - loss: 0.4141 - accuracy: 0.5769
Epoch 7/10
1921/1921 [==============================] - 7s 3ms/step - loss: 0.4134 - accuracy: 0.5769
Epoch 8/10
1921/1921 [==============================] - 6s 3ms/step - loss: 0.4162 - accuracy: 0.5768
Epoch 9/10
1921/1921 [==============================] - 7s 3ms/step - loss: 0.4155 - accuracy: 0.5768
Epoch 10/10
1921/1921 [==============================] - 9s 4ms/step - loss: 0.414

In [119]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

481/481 [==============================] - 1s 2ms/step - loss: 0.9675 - accuracy: 0.5742
Test set
  Loss: 0.968
  Accuracy: 0.574


In [222]:
new_plot ="After the members of a team of scientists (Harold Ramis, Dan Aykroyd, Bill Murray) lose their cushy positions at a university in New York City, they decide to become ghostbusters to wage a high-tech battle with the supernatural for money. They stumble upon a gateway to another dimension, a doorway that will release evil upon the city. The Ghostbusters must now save New York from complete destruction."

In [228]:
sep = tokenizer_pun.tokenize(new_plot)
sep = [w for w in sep if not w in stop_words]
sep = [stemmer.stem(w) for w in sep]
sep = [wordnet_lemmatizer.lemmatize(w) for w in sep]


In [229]:
seq = tokenizer.texts_to_sequences([sep])
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
get_genre_prediction((model.predict(padded) > 0.5).astype("int32"))

{'response': ['drama', 'thriller', 'horror']}

In [214]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [227]:
mode.save("genre_predictor.h5")

{'response': ['drama', 'thriller', 'horror']}